In [1]:
import time                                     #to use time.sleep()
from seleniumwire import webdriver              # using seleniumwire to allow inspect network requests/responses 
from seleniumwire.webdriver import ActionChains # using ActionChains to trigger .click() event on any kind of element

In [19]:
import json
import requests

In [59]:
#Set some env variables
postUrl = 'https://www.instagram.com/p/Bq-GLGsl30_/'
postHash = 'Bq-GLGsl30_' ####### GET IT AUTO LATER
queryHash = 'https://www.instagram.com/graphql/query/?query_hash='
matches = [postHash, queryHash, 'shortcode', 'first', 'after']

In [56]:
#using ChromeDriver
driver = webdriver.Chrome('./chromedriver.exe')
driver.get(postUrl);
print('Browser opening...')

Browser opening...


In [65]:
#class of 'load more' button -> glyphsSpriteCircle_add__outline__24__grey_9 u-__7
#getting the element based on his class
requestUrl = ''
element = driver.find_element_by_class_name('glyphsSpriteCircle_add__outline__24__grey_9.u-__7')
element.click()
for request in driver.requests:
    if all(x in request.url for x in matches):
        requestUrl = request.url

elements, requestUrl

([<selenium.webdriver.remote.webelement.WebElement (session="3db236a8e034358bee9f905710d709db", element="a2a56d11-b424-4354-9c62-cfade61c89e5")>],
 'https://www.instagram.com/graphql/query/?query_hash=bc3296d1ce80a24b1b6e40b1e72903f5&variables=%7B%22shortcode%22%3A%22Bq-GLGsl30_%22%2C%22first%22%3A12%2C%22after%22%3A%22QVFBWnFhRkVJT0NVd3Nnd0ZmUTR1WlhsRXJIVXhsRU1qeXAyLVNJc2RCQ2VlRUNsMDR2TUNKY3RIbjIxQXB5MEZNUl9CTXpPOU9GYjdUbEJTTjBvZUFVUw%3D%3D%22%7D')

In [66]:
#get the Hash to access the new comments page
r = requests.get(requestUrl)
nextPageHash = r.json()['data']['shortcode_media']['edge_media_to_parent_comment']['page_info']['end_cursor']

nextPageHash, r.json()

('QVFDZlpZVks2Y3B3NXRWeE1Hd3d1N2REcXlnQ3RrbWhrbzJpajNaZEI1OGd0Q1dRbWsxSzY2SW9oOFFuT0JCbTJyOEo3UGRKdWhMM0pfN1Z1TnJlUFlCTA==',
 {'data': {'shortcode_media': {'edge_media_to_parent_comment': {'count': 1110,
     'page_info': {'has_next_page': True,
      'end_cursor': 'QVFDZlpZVks2Y3B3NXRWeE1Hd3d1N2REcXlnQ3RrbWhrbzJpajNaZEI1OGd0Q1dRbWsxSzY2SW9oOFFuT0JCbTJyOEo3UGRKdWhMM0pfN1Z1TnJlUFlCTA=='},
     'edges': [{'node': {'id': '18134353945054026',
        'text': 'O cara do lado nao parece o patriota?',
        'created_at': 1586175637,
        'did_report_as_spam': False,
        'owner': {'id': '1669841239',
         'is_verified': False,
         'profile_pic_url': 'https://instagram.fsod2-1.fna.fbcdn.net/v/t51.2885-19/s150x150/54512186_2459931890903736_4575821821735075840_n.jpg?_nc_ht=instagram.fsod2-1.fna.fbcdn.net&_nc_ohc=c7PbqGsZ184AX8w9c3k&tp=1&oh=e95d07d57fd2799deba9c6dcd46b5a20&oe=5FF72A1C',
         'username': '_hebertreal_'},
        'viewer_has_liked': False,
        'edge_liked_b

In [ ]:
#get all comments from this page


In [ ]:
#closes browser
driver.quit()